In [1]:
from sklearn.linear_model import LinearRegression
from doubt import Boot
from doubt.datasets import PowerPlant


In [2]:
X, y = PowerPlant().split()
clf = Boot(LinearRegression())
clf.fit(X, y)
clf.predict([10, 30, 1000, 50], uncertainty=0.05)

(481.99688920651676, array([473.52247907, 490.16723009]))